# Jupyter (Javascript + Python) Widgets

The purpose of this application is to demonstrate how to
* send python variable and javascript to the browser
* get javascript variables back to python kernel

With Notebook compatible technology, using jupyter lab or notebook IDE and deployed as a Notebook and python module files (httpserver).

The **prerequisite** is that the jupyter server is enabled with **[jupyter server proxy](https://github.com/jupyterhub/jupyter-server-proxy)**.

It's fully validated using Jupyter Lab, and should work with 'minor' adaptation with classical notebooks

The steps are:

1. From the notebook, open an HTTP server, listening on port 8080 (could be randomized), able to serve 'get request' from the notebook
2. Use objects with _repr_javascript_ wich will call the above HTTP server
3. Wait the response which postion attributes in the **bag** object. This step is needed as the response is asynchronous

With this technology, we can, for example:

1. get the notebook url back in the python kernel
2. get the Geo position of the user (from the browser, requires a secured connection)
4. create an html5 range and monitor the value change from the user
3. more generally, get any variable accessible from the javascript's browser back to the python notebook

The 3 first use cases are demonstrated in this notebook

# Libraries

In [1]:
from httpserver import start, stop
from http.server import SimpleHTTPRequestHandler, HTTPServer
import os
from urllib.parse import urlparse, parse_qs
from IPython.display import clear_output
import json
import ipywidgets as widgets
import time

## Backend: the HTTP server

### Widgets python implementation

In [2]:
logsw=widgets.Textarea(layout=widgets.Layout(width='50%'))
logsw.value = ""
bag ={}
class myHTTPRequestHandler(SimpleHTTPRequestHandler):
    callback={}
    def __init__(self, *args, directory=None,bag=bag, **kwargs):
        self.bag = bag
        self.directory = os.path.join(os.getcwd(),"widgets")
        super().__init__(*args, directory=self.directory, **kwargs)
        # print(self.directory)
    def end_headers(self):
        super().end_headers()
    def do_GET(self):
        self.parsed_path = urlparse(self.path)
        self.queryparams = parse_qs(self.parsed_path.query)
        if self.path.endswith('/version'):
            self.version()
        elif self.parsed_path.path.endswith('location'):
            self.location()
        elif self.parsed_path.path.endswith('position'):
            self.position()
        elif self.parsed_path.path.endswith('_range'):
            self._range()
        else:
            super().do_GET()
    def version(self):
        ans = '{"version": "0.0"}'
        eans = ans.encode()        
        self.send_response(200)
        self.send_header("Content-type", "application/json")
        self.send_header("Content-Length",len(eans))
        self.end_headers()
        self.wfile.write(eans)
    def location(self):
        self.bag.update({self.queryparams['variable'][0]:self.queryparams['location'][0]})
        if 'location' in myHTTPRequestHandler.callback:
            self.callback['location'](self.queryparams['location'][0])
        self.version() 
    def position(self):
        if 'error' in self.queryparams:
            self.bag.update({self.queryparams['variable'][0]:"error"})
        else:
            self.bag.update({self.queryparams['variable'][0]:self.queryparams['position'][0]})
        if 'position' in self.callback:
            if 'error' in self.queryparams:
                self.callback['position']({"error":self.queryparams['error'][0]})
            else:
                self.callback['position'](self.queryparams['position'][0])
        self.version()
    def _range(self):
        self.bag.update({self.queryparams['variable'][0]:self.queryparams['value'][0]})
        if '_range' in self.callback:
            self.callback['_range'](self.queryparams['value'][0])
        self.version()
    def log_message(self, format, *args):
        global logsw
        v = format % args
        t = time.localtime()
        current_time = time.strftime("%H:%M:%S", t)
        logsw.value += current_time + " " + v +"\n"
logsw

Textarea(value='', layout=Layout(width='50%'))

In [3]:
start(handler_class=myHTTPRequestHandler)

Starting httpd...


## Frontend: Widgets javascript implementation

### Get calling url

Enable to get the server path and the notebook name

In [4]:
class Location(object):
    def __init__(self,variable):
        self.variable=variable
    def _repr_javascript_(self):
        """Return the code definitions and all invocations"""
        return f"""function jlb_location(variable) {{
        element.append(window.location); 
        fetch( window.location.origin+'/proxy/8080/location?location='+encodeURIComponent(window.location)+'&variable='+variable ) 
        .then(response => response.json())
        .then(data => console.log(data));;
    }}
    jlb_location('{self.variable}');
"""

In [5]:
loc = widgets.Label()
def located_cb(value):
    global loc
    loc.value = str(value)
location = Location('located')
myHTTPRequestHandler.callback.update({"location": located_cb})
loc

Label(value='')

In [6]:
location

### Get the user position

Requires a secured connection, else will return an error

In [7]:
fp = """function geoFindMe(variable) {
  function success(position) {
    const latitude  = position.coords.latitude;
    const longitude = position.coords.longitude;
    fetch( window.location.origin+'/proxy/8080/position?position='+encodeURIComponent(JSON.stringify({latitude:latitude,longitude:longitude}))+'&variable='+variable )
  }

  function error() {
    fetch( window.location.origin+'/proxy/8080/position?error=true&variable='+variable )
  }
  if(!navigator.geolocation) {
    fetch( window.location.origin+'/proxy/8080/position?error=Geolocation is not supported by your browser&variable='+variable );
  } else {
    navigator.geolocation.getCurrentPosition(success, error);
  }
}
"""
class Position(object):
    def __init__(self,variable):
        self.variable=variable
    def _repr_javascript_(self):
        return fp + f"geoFindMe('{self.variable}');"

In [8]:
pos = widgets.Label()
def positioned_cb(value):
    global pos
    pos.value = str(value)
position = Position('positioned')
myHTTPRequestHandler.callback.update({"position": positioned_cb})
pos

Label(value='')

In [9]:
position

### An html5 range

In [10]:
class Range(object):
    def __init__(self,variable):
        self.variable=variable
    def _repr_javascript_(self):
        return f"""window.rangeChange = function(range){{
            fetch( window.location.origin+'/proxy/8080/_range?value='+range.value+'&variable={self.variable}' )
        }}
        debugger;
        element.insertAdjacentHTML('beforeend','<input type="range" min="1" max="100" value="50" onchange="rangeChange(this)">');
        """
    # likely to be this.element[0] for legacy notebook but need some investigation

In [11]:
rng = widgets.Label()
def range_cb(value):
    global rng
    rng.value = str(value)
_range = Range('_range')
myHTTPRequestHandler.callback.update({"_range": range_cb})
output=widgets.Output()
with output:
    display(_range)
widgets.HBox([output,rng])

----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 57076)
Traceback (most recent call last):
  File "/usr/lib/python3.9/socketserver.py", line 683, in process_request_thread
    self.finish_request(request, client_address)
  File "/usr/lib/python3.9/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/tmp/ipykernel_648/1993679704.py", line 9, in __init__
    super().__init__(*args, directory=self.directory, **kwargs)
  File "/usr/lib/python3.9/http/server.py", line 653, in __init__
    super().__init__(*args, **kwargs)
  File "/usr/lib/python3.9/socketserver.py", line 747, in __init__
    self.handle()
  File "/usr/lib/python3.9/http/server.py", line 427, in handle
    self.handle_one_request()
  File "/usr/lib/python3.9/http/server.py", line 415, in handle_one_request
    method()
  File "/tmp/ipykernel_648/1993679704.py", line 19, in do_GET
    self.location()
  File

In [12]:
# stop()